<a href="https://colab.research.google.com/github/omerhac/arc_challenge/blob/master/arc_advanced_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import tensorflow as tf
import json
from google.cloud import storage
from matplotlib import pyplot as plt
from matplotlib import colors
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Conv2D, Lambda, Dense, Flatten, MaxPool2D, Input, BatchNormalization, Conv2DTranspose, UpSampling2D, Reshape
from sklearn.preprocessing import OneHotEncoder

AUTO = tf.data.experimental.AUTOTUNE

In [3]:
# get repository from github
!git clone https://github.com/omerhac/arc_challenge.git

Cloning into 'arc_challenge'...
remote: Enumerating objects: 134, done.
remote: Counting objects: 100% (134/134), done.
remote: Compressing objects: 100% (132/132), done.
Receiving objects: 100% (134/134), 10.49 MiB | 12.97 MiB/s, done.
remote: Total 134 (delta 55), reused 0 (delta 0), pack-reused 0
Resolving deltas: 100% (55/55), done.


In [4]:
# navigate to directory
cd arc_challenge

/content/arc_challenge


In [0]:
# load repository dependencies
import preprocess

In [0]:
## constants ##
BOARD_SIZE = (16,16) # board upperbound size
SN_BATCH_SIZE = 64
DECODER_BATCH_SIZE = 8
DENSE_REP_SIZE = 1024 # dense vector represantation size

# Load data / util funcitons


In [0]:
#@title load data
def load_data():
  """
  Loads all the data into training_tasks, eval_tasks and test_tasks
  """

  ## get paths
  GCS_PATH = "gs://kds-d3cfb3d523ca35d2517017a78110126404d01fdea69417ce49950459"
  training_filenames = tf.io.gfile.glob(GCS_PATH + "/training/*")
  test_filenames = tf.io.gfile.glob(GCS_PATH + "/test/*")
  eval_filenames = tf.io.gfile.glob(GCS_PATH + "/evaluation/*")

  # create datasets with filenames
  training_dataset = tf.data.Dataset.list_files(training_filenames)
  eval_dataset = tf.data.Dataset.list_files(eval_filenames)
  test_dataset = tf.data.Dataset.list_files(test_filenames)

  # load the jsons
  def load_task(filename):
    task_json = tf.io.read_file(filename)
    return task_json

  training_dataset = training_dataset.map(load_task)
  eval_dataset = eval_dataset.map(load_task)
  test_dataset = test_dataset.map(load_task)

  training_dataset_numpy = tf.data.Dataset.as_numpy_iterator(training_dataset) # convert to numpy iterator
  eval_dataset_numpy = tf.data.Dataset.as_numpy_iterator(eval_dataset)
  test_dataset_numpy = tf.data.Dataset.as_numpy_iterator(test_dataset)

  ## create a numpy array of tasks (n_tasks, )
  def list_from_jsons(jsons_numpy_iterator):
    """
      Create a list of task dictionaries from jsons numpy interator
    """

    tasks = []
    for task in jsons_numpy_iterator:
      tasks.append(json.loads(task))

    return tasks

  ## get numpy arrays of datasets
  training_tasks = list_from_jsons(training_dataset_numpy)
  eval_tasks = list_from_jsons(eval_dataset_numpy)
  test_tasks = list_from_jsons(test_dataset_numpy)

  return training_tasks, eval_tasks, test_tasks


def load_data_from_jsons():
  """
  Load tasks from jsons to lists of tasks.

  Returns:
  training_tasks, eval_tasks, test_tasks --> lists of tasks
  """

  with open("training_tasks.json", 'r') as f:
    training_tasks = json.load(f)

  with open("eval_tasks.json", 'r') as f:
    eval_tasks = json.load(f)
  
  with open("test_tasks.json", 'r') as f:
    test_tasks = json.load(f)

  return training_tasks, eval_tasks, test_tasks

In [8]:
%%time
training_tasks, eval_tasks, test_tasks = load_data_from_jsons()

CPU times: user 232 ms, sys: 16.2 ms, total: 248 ms
Wall time: 249 ms


## Utility functions

In [0]:
#@title Utilities
def plot_board(board, ax, title=""):
  """
  Plot a board on a given axis
  """
  cmap = colors.ListedColormap(
      ['#000000', '#0074D9','#FF4136','#2ECC40','#FFDC00',
        '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'])
  norm = colors.Normalize(vmin=0, vmax=9)
  
  ax.imshow(board, cmap=cmap, norm=norm)
  ax.grid(True,which='both',color='lightgrey', linewidth=0.5)    
  ax.set_yticks([x-0.5 for x in range(1+board.shape[0])])
  ax.set_xticks([x-0.5 for x in range(1+board.shape[1])])     
  ax.set_xticklabels([])
  ax.set_yticklabels([])
  ax.set_title(title)

def plot_one(task, ax, i,train_or_test,input_or_output):
  """
  Plot one task on a given axis
  """
  cmap = colors.ListedColormap(
      ['#000000', '#0074D9','#FF4136','#2ECC40','#FFDC00',
        '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'])
  norm = colors.Normalize(vmin=0, vmax=9)
  
  input_matrix = task[train_or_test][i][input_or_output]
  ax.imshow(input_matrix, cmap=cmap, norm=norm)
  ax.grid(True,which='both',color='lightgrey', linewidth=0.5)    
  ax.set_yticks([x-0.5 for x in range(1+len(input_matrix))])
  ax.set_xticks([x-0.5 for x in range(1+len(input_matrix[0]))])     
  ax.set_xticklabels([])
  ax.set_yticklabels([])
  ax.set_title(train_or_test + ' '+input_or_output)
    

def plot_task(task):
    """
    Plots the first train and test pairs of a specified task,
    using same color scheme as the ARC app
    """    
    num_train = len(task['train'])
    fig, axs = plt.subplots(2, num_train, figsize=(3*num_train,3*2))
    for i in range(num_train):     
        plot_one(task, axs[0,i],i,'train','input')
        plot_one(task, axs[1,i],i,'train','output')        
    plt.tight_layout()
    plt.show()        
        
    num_test = len(task['test'])
    fig, axs = plt.subplots(2, num_test, figsize=(3*num_test,3*2))
    if num_test==1: 
        plot_one(task, axs[0],0,'test','input')
        plot_one(task, axs[1],0,'test','output')     
    else:
        for i in range(num_test):      
            plot_one(task, axs[0,i],i,'test','input')
            plot_one(task, axs[1,i],i,'test','output')  
    plt.tight_layout()
    plt.show() 
  

def plot_board_pairs(board_pairs, labels):
  """
  Plots the board pairs (for siamese networks) with their label as a title
  """

  fig, axs = plt.subplots(len(board_pairs), 2, figsize=(8, 3 * len(board_pairs)))
  
  for i, pair in enumerate(board_pairs):
    # plot a pair on a given axis
    plot_board(pair[0], axs[i, 0], title="anchor") 
    plot_board(pair[1], axs[i, 1], title=labels[i])
  
  plt.tight_layout()
  plt.show()


def plot_decoder_boards(board_pairs):
  """
  Plots the board pairs (for siamese networks) with their label as a title
  """

  fig, axs = plt.subplots(len(board_pairs), 2, figsize=(8, 3 * len(board_pairs)))

  for i, pair in enumerate(board_pairs):
    # plot a pair on a given axis
    plot_board(pair[0], axs[i, 0]) 
    plot_board(pair[1], axs[i, 1])

  plt.tight_layout()
  plt.show()


def display_training_curves(hist, metric='accuracy', with_val=False):
  """display learning curves for keras history dict, args: history dict, with val --> boolean with/without val"""
  plt.figure(figsize=(18,6))

  # accuracy plots
  plt.subplot(1,2,1)
  plt.plot(hist[metric])
  
  if with_val:
    plt.plot(hist['val_' + metric])
    plt.legend(['Train', 'Validation'])
  
  else:
    plt.legend(['Train'])
  
  plt.title('Model accuracy')
  plt.xlabel('EPOCH')
  plt.ylabel('Accuracy')

  # loss plots
  plt.subplot(1,2,2)
  plt.plot(hist['loss'])

  if with_val:
    plt.plot(hist['val_loss'])
    plt.legend(['Train loss', 'Val loss'])
  
  else:
    plt.legend(['Train loss'])
  
  plt.title('Model loss')
  plt.xlabel('EPOCH')
  plt.ylabel('Loss')
  plt.show()

# Data generating functions

In [0]:
def get_dataset_shapes(dataset):
  """
  Returns dataset board shapes. 
  """

  shape_0 = []
  shape_1 = []
  
  # check every task
  for task in dataset:
    boards = get_task_boards(task) # get all boards
    shape_0 += [board.shape[0] for board in boards]
    shape_1 += [board.shape[1] for board in boards]
  
  return shape_0, shape_1

def get_task_boards(task, threshold_shape=(40, 40), pad=None, test=False, divide_sets=False):
  """
  Get the training / testing boards of every example in a specific task. 

  Args: threshold_shape --> threshold shape for which 
  biggger samples won't be returned. 
  pad --> functional padding function to pad boards with (up tp threshold_size).
  test --> bool, whether a test task or not. for output of task example.
  divide_sets --> bool, whether to divide boards sets to training - input / output, test - input / output.
  """
  
  training_input_boards, training_output_boards, test_input_boards, test_output_boards = [], [], [], []

  # train boards
  for example in task['train']:
    input_board = np.array(example['input'])
    output_board = np.array(example['output'])

    if((input_board.shape[0] < threshold_shape[0]) and (input_board.shape[1] < threshold_shape[1])):
      training_input_boards.append(pad(input_board, output_shape=threshold_shape) if pad else input_board) # check for padding func

    if((output_board.shape[0] < threshold_shape[0]) and (output_board.shape[1] < threshold_shape[1])):
      training_output_boards.append(pad(output_board, output_shape=threshold_shape) if pad else output_board) # check for padding func

  # test boards
  for example in task['test']:
    input_board = np.array(example['input'])

    if not test: # check if test example
      output_board = np.array(example['output'])
    
    # check whether the board is smaller then threshold shape
    if((input_board.shape[0] < threshold_shape[0]) and (input_board.shape[1] < threshold_shape[1])):
      test_input_boards.append(pad(input_board, output_shape=threshold_shape) if pad else input_board) # check for padding func

    if((output_board.shape[0] < threshold_shape[0]) and (output_board.shape[1] < threshold_shape[1]) and (not test)):
      test_output_boards.append(pad(output_board, output_shape=threshold_shape) if pad else output_board) # check for padding func
  
  # whether to divide boards:
  if divide_sets:
    return training_input_boards, training_output_boards, test_input_boards, test_output_boards
  else:
    return training_input_boards + training_output_boards + test_input_boards + test_output_boards

def pad(mat, output_shape, padder=0):
  """
  Pad a matrix with padder up to output_shape. Insert matrix at upper left corner.
  
  Args:
  mat - np.array matrix of rank 2
  output_shape - tuple 
  padder - int
  """

  output_board = np.zeros(shape=output_shape) + padder # create output board and pad it

  # get input shape
  input_rows = mat.shape[0]
  input_cols = mat.shape[1]

  # if random=False, insert input matrix in upper left corner
  output_board[:input_rows, :input_cols] = mat
  return output_board

def random_pad(mat, output_shape, padder=0):
  """
  Pad a matrix with padder up to output_shape. Insert the matrix at a random location
  
  Args:
  mat - np.array matrix of rank 2
  output_shape - tuple 
  padder - int
  seed - int
  """

  output_board = np.zeros(shape=output_shape) + padder # create output board and pad it

  # get input shape
  input_rows = mat.shape[0]
  input_cols = mat.shape[1]

  # insert mat at a random loacation
  # get random location
  start_row = np.random.randint(output_shape[0] - input_rows)
  start_col = np.random.randint(output_shape[1] - input_cols)
  # insert
  output_board[start_row:start_row+input_rows, start_col:start_col+input_cols] = mat

  return output_board

def get_all_boards(training_tasks, eval_tasks, test_tasks):
  """
  Extracts all the boards from all training/eval/testing tasks. 
  """

  training_boards = []
  for task in training_tasks:
    training_boards += get_task_boards(task,threshold_shape=BOARD_SIZE, pad=pad)

    # augment training set (random pad)
    training_boards += get_task_boards(task,threshold_shape=BOARD_SIZE, pad=random_pad)
    training_boards += get_task_boards(task,threshold_shape=BOARD_SIZE, pad=random_pad)

  eval_boards = []
  for task in eval_tasks:
    eval_boards += get_task_boards(task,threshold_shape=BOARD_SIZE, pad=pad)

    # augment eval set (random pad)
    eval_boards += get_task_boards(task,threshold_shape=BOARD_SIZE, pad=random_pad)
    eval_boards += get_task_boards(task,threshold_shape=BOARD_SIZE, pad=random_pad)
    

  test_boards = []
  for task in test_tasks:
    test_boards += get_task_boards(task,threshold_shape=BOARD_SIZE, pad=pad, test=True)

    # augment test set (random pad)
    test_boards += get_task_boards(task,threshold_shape=BOARD_SIZE, pad=random_pad, test=True)
    test_boards += get_task_boards(task,threshold_shape=BOARD_SIZE, pad=random_pad, test=True)

  all_boards = training_boards+eval_boards+test_boards
  return all_boards



## Data augmentation

In [0]:
def get_rotated_views(board):
  """
  Turns a board 90 deg counter clockwise 3 times. Returns a list length 4 with all the rotated views including the original one.
  """

  rotates = [board]
  for i in range(3):
    board = np.rot90(board)
    rotates.append(board)
  
  return rotates

def get_rotated_data_pairs(rotated_views):
  """
  Creates all the possible pairs out of the rotated views tensor and labels them. If a1..a4 are rotated vies (A.C.W) of a1, returns labels according to:
   - (a1, a1) = 1 --> same board
   - (a1, a2) = 2 --> 90 a.c.w rotate
   - (a1, a3) = 3 --> 180 rotate
   - (a1, a4) = 4 --> 270 a.c.w rotate
   - ...
   - (a3, a4) = 2 --> 90 a.c.w rotate

  Args:
  rotated_views = a list of 4 rotated views of the board

  Returns:
  pairs: a list of tuples of boards
  labels: a list of labels matching each pair
  """

  # stopping rule
  if rotated_views == []:
    return [], []

  anchor = rotated_views[0]  # select board to compare with 
  pairs, labels = [], []
  label = 1 # init label

  # iterate over all remaining examples
  for view in rotated_views:
    pairs.append((anchor, view))
    labels.append(label)
    label += 1 # update label, views rotate a.c.w
  
  next_pairs, next_labels = get_rotated_data_pairs(rotated_views[1:]) # recursive call

  return pairs + next_pairs, labels + next_labels

def get_binary_board(board):
  """
  Returns a binary board. Every non 0 value becomes 1. 0 stays 0.
  """

  return (board != 0).astype('int32')


## Create dataset

In [0]:
def model_shape_board(board):
  """
  Reshapes board for moedl digestion. shape [*BOARD_SIZE, 1]
  """
  return board.reshape([*BOARD_SIZE, 1])

def plotting_shape_board(board):
  """
  Reshapes the board for plotting. shape [*BOARD_SIZE]
  """
  return board.reshape([*BOARD_SIZE])

def get_all_pairs_reshaped(board, all_boards):
  """
  Creates a list of pairs of data for the board and reshapes the boards to shape [*BOARD_SHAPE, 1] for nn digestion. For all the possible rotates and a false match

  Args:
  board - np.array
  all_boards - all the other boards on the dataset not including this one

  Returns:
  a list of 5 tuples, 4 for the rotation part and 1 for a false match
  """

  rotated_views = get_rotated_views(board)
  rotated_pairs, rotated_labels = get_rotated_data_pairs(rotated_views)
  rotated_pairs = [(model_shape_board(t[0]), model_shape_board(t[1])) for t in rotated_pairs] # reshape boards

  # create false match pair
  different_board = all_boards[np.random.randint(len(all_boards))] # generate random example
  false_label = 0

  return rotated_pairs + [[board.reshape([*BOARD_SIZE, 1]), different_board.reshape([*BOARD_SIZE, 1])]], rotated_labels + [false_label]

def get_dataset_from_lists(pair_list, label_list, for_encoder=False):
  """
  Creates an x,y dataset from a list of pairs and a list of labels. One hot encode labels

  Args:
  pair list: a list of pairs of images
  label list: a list of ints - labels
  for_encoder: bool - determains if the dataset if for the encoder

  Retrurns:
  x - [x1, x2] a list of two numpy array, each of shape [n_samples, *BOARD_SHAPE, 1] containing stacked inputs of one of the siamese twins OR only x1 if for encoder
  y - one hot labels, only for siamese networks
  """

  # prepare x
  pairs_stacked = np.stack(pair_list)
  x1 = pairs_stacked[:,0,:,:] # get one column of inputs
  x2 = pairs_stacked[:,1,:,:] # get the second one
  x = [x1, x2]

  # prepare y
  ohe = OneHotEncoder(sparse=False) # sparse = false is super importent!!
  stacked_labels = np.stack(label_list).reshape([-1,1]) # stack and reshape label
  y = ohe.fit_transform(stacked_labels)

  # if the dataset is for the decoder return only one column of images
  if not for_encoder:
    return x, y
  
  else:
    return x1
  

def normalize_boards(boards):
  """
  Normalize a list of boards. The boards already have unit variance so we only need to zero-mean them.
  """
  
  sum = 0

  # check what is the avarage of the boards:
  for board in boards:
    sum += board.sum()
  
  avarage = sum / (len(boards) * BOARD_SIZE[0] * BOARD_SIZE[1])
  
  # normalize boards
  norm_boards = [board - avarage for board in boards]

  return norm_boards

In [0]:
def get_siamese_dataset(training_tasks, eval_tasks, test_tasks, get_lists=False):
  """
  Creates a dataset for the siamese networks.

  Args:
  .._tasks: list of tasks.
  get_lists: bool, whether to get just the lists of pairs or the processed dataset.
  """

  # extract all_boards
  all_boards = get_all_boards(training_tasks, eval_tasks, test_tasks)

  # binirize all boards
  all_boards_binary = [get_binary_board(board) for board in all_boards]

  # normalize boards --> not sure if necessary
  #all_boards_binary = normalize_boards(all_boards_binary)

  # create a list of all boards augmentation data
  pair_list = []
  label_list = []

  # iterate over all boards
  for i, board in enumerate(all_boards_binary):
    board_pairs, board_labels = get_all_pairs_reshaped(board, all_boards[i:]) # augment example. use only boards from here onward
    pair_list += board_pairs
    label_list += board_labels

  ## create dataset
  x, y = get_dataset_from_lists(pair_list, label_list)

  if get_lists:
    return pair_list, label_list

  else:
    return x, y

In [0]:
### just for this notebook
x, y = get_siamese_dataset(training_tasks, eval_tasks, test_tasks)

# Siamese networks architecture
- getting the right architecture
- went for residual networks

## Defining basic blocks

In [0]:
#@title basic blocks
def residual_encoder_block(filter_num, kernel_size, bn_moment):
  """
  A functional style residual connection convolutional block.
  """

  def block(x, filter_num, kernel_size=(3,3), bn_moment=0.9):
    # first layer
    c1 = Conv2D(filter_num, kernel_size=kernel_size, activation='relu', padding='same')(x)
    c1 = BatchNormalization(momentum=bn_moment)(c1)

    # second layer
    c2 = Conv2D(filter_num, kernel_size=kernel_size, activation='relu', padding='same')(c1)
    c2 = BatchNormalization(momentum=bn_moment)(c2)

    # third layer
    c3 = Conv2D(filter_num, kernel_size=kernel_size, activation='relu', padding='same')(c2)
    c3 = BatchNormalization(momentum=bn_moment)(c3)

    # residual connection
    res = c1 + c3

    return res
  
  return lambda x: block(x, filter_num, kernel_size, bn_moment)

def residual_decoder_block(filter_num, kernel_size, bn_moment, first=False):
  """
  A functional style residual connection deconvolutional block.

  Args:
  first - bool, determains if it is the first block of a network
  """
  
  def block(x, filter_num, kernel_size=(3,3), bn_moment=0.9):
    
    # first layer
    if first:
      dc1 = Dense(4*4*filter_num, activation='linear')(x) # num neurons is dependent upon the number of blocks
      dc1 = BatchNormalization(momentum=bn_moment)(dc1)
      dc1 = Reshape(target_shape=(4,4,filter_num))(dc1) # reshaping is dependent upon the number of blocks
    
    else:
      dc1 = Conv2DTranspose(filter_num, kernel_size=kernel_size, activation='relu', padding='same')(x)
      dc1 = BatchNormalization(momentum=bn_moment)(dc1)

    # second layer
    dc2 = Conv2DTranspose(filter_num, kernel_size=kernel_size, activation='relu', padding='same')(dc1)
    dc2 = BatchNormalization(momentum=bn_moment)(dc2)

    # third layer
    dc3 = Conv2DTranspose(filter_num, kernel_size=kernel_size, activation='relu', padding='same')(dc2)
    dc3 = BatchNormalization(momentum=bn_moment)(dc3)

    # residual connection
    res = dc1 + dc3

    return res

  return lambda x: block(x, filter_num, kernel_size, bn_moment)

## Main architecture

In [0]:
def get_siamese_networks_model(input_shape):
  """
  Creates siamese networks model. ref paper: https://www.cs.cmu.edu/~rsalakhu/papers/oneshot1.pdf
  """

  # define input vectors
  input1 = Input(input_shape, name='input1')
  input2 = Input(input_shape, name='input2')

  def get_encoder():
    """
    A helper function for sharing weights between inputs
    """

    x = Input(input_shape, name='x')  

    # first block
    block1 = residual_encoder_block(16, kernel_size=(3,3), bn_moment=0.9)(x)
    mp1 = MaxPool2D(pool_size=(2,2), name='max_pool1')(block1)

    # second block
    block2 = residual_encoder_block(32, kernel_size=(3,3), bn_moment=0.9)(mp1)
    mp2 = MaxPool2D(pool_size=(2,2), name='max_pool2')(block2)

    # third block
    block3 = residual_encoder_block(64, kernel_size=(3,3), bn_moment=0.9)(mp2)
    mp3 = MaxPool2D(pool_size=(2,2), name='max_pool3')(block3)

    # flatten
    flat = Flatten()(mp3)

    # dense
    dense = Dense(DENSE_REP_SIZE, activation='sigmoid', name='dense_rep')(flat)
    bn_dense = BatchNormalization(momentum=0.9)(dense)

    return tf.keras.Model(inputs=x, outputs=bn_dense)

  # get shared encoder
  encoder = get_encoder()
  
  # get feature vectors
  v1 = encoder(input1)
  v2 = encoder(input2)

  # compute L1 loss
  L1_Layer = Lambda(lambda tensors: tf.abs(tensors[0] - tensors[1]))
  L1_diff = L1_Layer([v1, v2])

  # compute probs
  probs = Dense(5, activation='softmax')(L1_diff)

  siamese_net = tf.keras.Model(inputs=[input1, input2], outputs=probs)

  # compile
  siamese_net.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
  return siamese_net

## Training siamese networks

In [0]:
sn = get_siamese_networks_model([*BOARD_SIZE, 1])

In [74]:
hist = sn.fit(x=x, y=y, epochs=30, batch_size=SN_BATCH_SIZE, shuffle=True, validation_split=0.2)

Epoch 1/30
2343/2343 [==============================] - 47s 20ms/step - loss: 0.0512 - categorical_accuracy: 0.9843 - val_loss: 0.6027 - val_categorical_accuracy: 0.8662
Epoch 2/30
2343/2343 [==============================] - 47s 20ms/step - loss: 0.0372 - categorical_accuracy: 0.9885 - val_loss: 0.8198 - val_categorical_accuracy: 0.8888
Epoch 3/30
2343/2343 [==============================] - 47s 20ms/step - loss: 0.0361 - categorical_accuracy: 0.9889 - val_loss: 0.8168 - val_categorical_accuracy: 0.8938
Epoch 4/30
2343/2343 [==============================] - 47s 20ms/step - loss: 0.0311 - categorical_accuracy: 0.9901 - val_loss: 0.5342 - val_categorical_accuracy: 0.8938
Epoch 5/30
2343/2343 [==============================] - 48s 21ms/step - loss: 0.0271 - categorical_accuracy: 0.9911 - val_loss: 0.4717 - val_categorical_accuracy: 0.8829
Epoch 6/30
2343/2343 [==============================] - 47s 20ms/step - loss: 0.0241 - categorical_accuracy: 0.9923 - val_loss: 0.4898 - val_categoric

KeyboardInterrupt: ignored

In [0]:
display_training_curves(hist.history, metric='categorical_accuracy', with_val=True)

# Board generator (decoder) architecture

In [0]:
def get_encoder_from_siamese(siamese):
  """
  Creates a board encoder from the siamese networks model. Maps board --> feature vector of dim 512
  """

  # create encoder from all layers up to dense
  model = tf.keras.models.Sequential([
                                      tf.keras.layers.InputLayer(input_shape=[*BOARD_SIZE, 1]),
                                      siamese.layers[2] # all sequential layers from siamese networks model
  ])

  return model

def get_decoder(siamese):
  """
  Creates a decoder from siamese netwok model. Maps feature vector of dim 512 --> board. 
  Same architecture from encoder is preserved
  """

  # build model
  inp = Input([DENSE_REP_SIZE])

  # first_block
  block1 = residual_decoder_block(64, kernel_size=(3,3), bn_moment=0.9, first=True)(inp)
  us1 = UpSampling2D(size=(2,2))(block1)

  block2 = residual_decoder_block(32, kernel_size=(3,3), bn_moment=0.9)(us1)
  us2 = UpSampling2D(size=(2,2))(block2)

  block3 = residual_decoder_block(16, kernel_size=(3,3), bn_moment=0.9)(us2)

  output = Conv2D(1, kernel_size=(1,1), activation='sigmoid')(block3)

  model = tf.keras.Model(inputs=inp, outputs=output)

  # compile
  model.compile(loss=pixelwise_error_loss, optimizer='adam', metrics=[pixelwise_auc])

  return model


def pixelwise_error_loss(y_true, y_pred):
  """
  Custom loss function. Pixelwise error from the true image. ||y_true - y_pred||
  """

  return tf.keras.backend.sum((y_true - y_pred) ** 2) / y_pred.shape[0]


def pixelwise_accuracy(y_true, y_pred, threshold=0.8):
  """
  Custom accuracy function. Pixelwise accuracy of predicting 0 / 1
  """
  
  tp = tf.math.reduce_sum(tf.math.multiply(tf.cast((y_pred >= threshold), 'float32'), y_true)) # cast boolean to binary
  tn = tf.math.reduce_sum(tf.math.multiply(tf.cast((y_pred < threshold), 'float32'), tf.cast((y_true == 0), 'float32')))
  n_predictions = BOARD_SIZE[0] * BOARD_SIZE[1]

  return ((tp + tn) / n_predictions) / y_pred.shape[0]


def pixelwise_sensitivity(y_true, y_pred, threshold=0.8):
  """
  Pixelwise sensitivity of correctly predicting 1s. true positives / positives
  """

  tp = tf.math.reduce_sum(tf.math.multiply(tf.cast((y_pred >= threshold), 'float32'), y_true)) 
  p = tf.cast(tf.math.reduce_sum(y_true), 'float32') 
  
  return tf.cond(p > 0, lambda: tp / p, lambda: tf.constant(1, dtype='float32')) # don't divide by zero

def pixelwise_fpr(y_true, y_pred, threshold=0.8):
  """
  Pixelwise false positive rate of wrongly predicting 1s. false positives / negatives
  """

  fp = tf.math.reduce_sum(tf.math.multiply(tf.cast((y_pred >= threshold), 'float32'), tf.cast((y_true==0), 'float32'))) # cast boolean to binary
  n = tf.cast(tf.math.reduce_sum(tf.cast((y_true == 0), 'float32')), 'float32')

  return tf.cond(n > 0, lambda: fp / n, lambda: tf.constant(1, dtype='float32')) # don't divide by zero


def pixelwise_auc(y_true, y_pred, num_thresholds=10):
  """
  Pixelwise auc score. 
  """

  sensitivity = []
  fpr = []

  # compute graph points
  for threshold in np.linspace(1, 0, num=num_thresholds):
    sensitivity.append(pixelwise_sensitivity(y_true, y_pred, threshold=threshold))
    fpr.append(pixelwise_fpr(y_true, y_pred, threshold=threshold))
  
  # compute trapeze area
  trapeze = []
  for i in range(len(sensitivity)- 1):
    area = tf.multiply(sensitivity[i], fpr[i+1] - fpr[i])
    trapeze.append(area)
  
  auc_score = tf.add_n(trapeze) 

  return auc_score

In [0]:
dec = get_decoder(sn)

## Decoder dataset


In [0]:
def get_decoder_dataset(siamese, encoder_dataset):
  """
  Creates the decoder dataset from the encoder.
  
  Args:
  siamese - siamese model to extract encoder from
  encoder dataset - boards to encoder to feature vectors

  Returns:
  x - boards feature vectors, predicted by encoder
  y - boards 
  """
  
  enc = get_encoder_from_siamese(siamese) # fetch encoder

  # build dataset
  x = enc.predict(encoder_dataset) # get feature vectors
  y = encoder_dataset # boards to recreate

  return x,y

# create encoder dataset from the old pair and label lists
pair_list, label_list = get_siamese_dataset(training_tasks, eval_tasks, test_tasks, get_lists=True)
encoder_dataset = get_dataset_from_lists(pair_list, label_list, for_encoder=True) 
# create decoder dataset from encoder
decoder_x, decoder_y = get_decoder_dataset(sn, encoder_dataset)

In [0]:
hist_decoder = dec.fit(x=decoder_x, y=decoder_y, batch_size=DECODER_BATCH_SIZE, shuffle=True, epochs=1, validation_split=0.2)

 1918/18744 [==>...........................] - ETA: 2:30 - loss: 17.4123 - pixelwise_auc: 0.8597

# Check regular encoder-decoder architecture
## For checking some hyperparameters and deciding which adecoder architecture to use

## Some takeaways:
- adding two sets of randomly padded data helps alot. more doesnt help
- small decoder batch size (4-16) really helps
- adding more neurons to the compressed represantation (512 --> 1024) really helps
- removing the last layers batch normalization really helps
 - removing bn before compressed representation isnt good
- penelizing mistaking 1's for 0's more doesnt help at all
- changing last layer activation to linear and clipping to [0,1] helps to some extent
- chagning middle to relu - really bad!!!
- addind another conv-deconv layer at the beggining helps a little
- adding another conv-deconv layer at the end doesnt help at all
- residual layers - really cool! used the architecture from this papare -- >https://github.com/omerhac/arc_challenge/blob/master/deep%20residual%20conv-deconv%20network.pdf
- more types of data augmentation....


In [0]:
#@title encoder-decoder baseline
def get_encoder_decoder():
  """
  Create an encoder decoer "normal" architecture, with residual connections
  """
  inp = tf.keras.layers.Input([*BOARD_SIZE, 1])
  
  # encoder
  block1 = residual_encoder_block(16, kernel_size=(3,3), bn_moment=0.9)(inp)
  mp1 = MaxPool2D(pool_size=(2,2))(block1)

  block2 = residual_encoder_block(32, kernel_size=(3,3), bn_moment=0.9)(mp1)
  mp2 = MaxPool2D(pool_size=(2,2))(block2)

  block3 = residual_encoder_block(64, kernel_size=(3,3), bn_moment=(0.9))(mp2)

  # flatten
  flat = Flatten()(block3)

  # dense representation
  dense = Dense(1024, activation='sigmoid')(flat)
  dense = BatchNormalization(momentum=0.9)(dense)


  # decoder 

  # first_block
  block1 = residual_decoder_block(64, kernel_size=(3,3), bn_moment=0.9, first=True)(dense)
  us1 = UpSampling2D(size=(2,2))(block1)

  block2 = residual_decoder_block(32, kernel_size=(3,3), bn_moment=0.9)(us1)
  us2 = UpSampling2D(size=(2,2))(block2)

  block3 = residual_decoder_block(16, kernel_size=(3,3), bn_moment=0.9)(us2)

  output = Conv2D(1, kernel_size=(1,1), activation='sigmoid')(block3)

  model = tf.keras.Model(inputs=inp, outputs=output)
  

  # compile
  model.compile(loss=pixelwise_error_loss, optimizer='adam', metrics=[pixelwise_auc])
  return model

In [0]:
ed = get_encoder_decoder()

In [0]:
ed_hist = ed.fit(x=decoder_y, y=decoder_y, batch_size=8, shuffle=True, epochs=20, validation_split=0.2) # get image --> predict image

In [0]:
## check predictions

In [0]:
boards = decoder_y[-10:]

# predict
predictions = ed.predict(boards)

# reshape
boards = [plotting_shape_board(board) for board in boards]
predictions = [plotting_shape_board(board) for board in predictions]
pairs = zip(boards, predictions)

plot_decoder_boards(list(pairs))


# Interleaved training (decoder/encoder/deocoder..)

In [0]:
def copy_decoder_to_siamese(siamese, decoder):
  """
  Copys the weights from the decoder to the siamese networks model
  """

  decoder_layers = decoder.layers # get decoder layers
  siamese_layers = siamese.layers[2].layers # siamese encoder layers

  for i, layer in enumerate(decoder_layers[::-1]):
      if(2 >= len(layer.weights) > 0): # trainable layer and not BN layer
        if layer.weights[0].shape[0] < 100: # deconv layers
          decoder_layer_weigths = layer.get_weights() # get weights from decoder
          bias = np.zeros(siamese_layers[i].get_weights()[1].shape) # init new bias
          w = decoder_layer_weigths[0].transpose([1,0,2,3]) # transpose weights
          siamese_layers[i].set_weights([w, bias]) # set siamese weights

        else:
          # dense layer
          decoder_layer_weigths = layer.get_weights() # get weights from decoder
          bias = np.zeros(siamese_layers[i].get_weights()[1].shape) # init new bias
          w = decoder_layer_weigths[0].transpose() # transpose weights
          siamese_layers[i].set_weights([w, bias]) # set siamese weights


def copy_siamese_to_decoder(siamese, decoder):
  """
  Copys the weights from the siamese networks model to the decoder
  """

  decoder_layers = decoder.layers # get decoder layers
  siamese_layers = siamese.layers[2].layers # siamese encoder layers

  for i, layer in enumerate(decoder_layers[::-1]):
      if(2 >= len(layer.weights) > 0): # trainable layer and not BN layer
        if layer.weights[0].shape[0] < 100: # deconv layers
          siamese_layer_weigths = siamese_layers[i].get_weights() # get weights from siamese encoder
          bias = np.zeros(layer.get_weights()[1].shape) # init new bias
          w = siamese_layer_weigths[0].transpose([1,0,2,3]) # transpose weights
          layer.set_weights([w, bias]) # set decoder weights

        else:
          # dense layer
          siamese_layer_weigths = siamese_layers[i].get_weights() # get weights from siamese encoder
          bias = np.zeros(layer.get_weights()[1].shape) # init new bias
          w = siamese_layer_weigths[0].transpose() # transpose weights
          layer.set_weights([w, bias]) # set decoder weights


In [0]:
# create new models with same weights
EPOCHS = 30
sn = get_siamese_networks_model([*BOARD_SIZE, 1])
decoder = get_decoder(sn, copy_encoder_weights=False)

In [0]:
from tqdm import tqdm
SN_BATCH_SIZE = 64
DEC_BATCH_SIZE = 4
DATA_SPLIT = 5
SN_STEPS_PER_EPOCH = len(x) // SN_BATCH_SIZE
DEC_STEPS_PER_EPOCH = len(decoder_x) // DEC_BATCH_SIZE

# even fancier training loop
for i in range(EPOCHS):
  print("training... epoch num: {}".format(i))

  for i in range(40):
    # train siamese
    _ = sn.fit(x=x, y=y, epochs=1, batch_size=SN_BATCH_SIZE, shuffle=True, steps_per_epoch=(780//DATA_SPLIT))
    
    # copy weights
    copy_siamese_to_decoder(sn, decoder)

    # get new decoder dataset
    decoder_x, decoder_y = get_decoder_dataset(sn, encoder_dataset) # encoder dataset is the same as before

    # train decoder
    _ = decoder.fit(x=decoder_x, y=decoder_y, epochs=1, batch_size=DEC_BATCH_SIZE, shuffle=True, steps_per_epoch=(24000//DATA_SPLIT)) #### WOW!!! use smaller batch size, WOHOOO!!

    # copy weights
    copy_decoder_to_siamese(sn, decoder)

# Predicting from diffrance in boards

In [0]:
task_training_input, task_training_output, task_test_input, task_test_output = get_task_boards(training_tasks[0], pad=pad, divide_sets=True)

In [0]:
# toy dataset (x is the dataset for the siamese networks)
anchor = x[0][0]
rotate_once_anchor = x[1][1]
rotate_twice_anchor = x[1][2]

toy = np.stack([anchor, rotate_once_anchor, rotate_twice_anchor])

# plot
fig, axs = plt.subplots(1,3)
plot_board(plotting_shape_board(anchor), axs[0])
plot_board(plotting_shape_board(rotate_once_anchor), axs[1])
plot_board(plotting_shape_board(rotate_twice_anchor), axs[2])

## define predictor

In [0]:
def get_predictor(decoder):
  """
  Builds a detector which takes a board and a rules vector and predicts output board.

  Args:
  decoder --> trained decoder 
  """
  